In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)
import os
os.chdir('..')

# 内存保存历史 + 窗口记忆

In [2]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

model = ChatOpenAI()
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一个数学老师。"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)
chain = prompt | model | StrOutputParser()

定义记忆管理器：

In [3]:
from langchain_chinese import MemoryManager
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory

window = ConversationBufferWindowMemory(
  return_messages=True, k=2, chat_memory = ChatMessageHistory()
)

memory = MemoryManager(shorterm_memory = window)

In [4]:
from langchain_chinese import WithMemoryBinding

withMemoryChain = WithMemoryBinding(
  chain,
  memory,
  input_messages_key="input",
  history_messages_key="history",
)

In [5]:
withMemoryChain.invoke(
  {"input": "三角函数什么意思？?"},
  config={"configurable": {"session_id": "abc123"}},
)

'三角函数是指正弦、余弦、正切、余切、正割和余割这六个与直角三角形内角相关的函数。这些函数是数学中重要的概念，用于描述角度和三角形之间的关系。在数学中，三角函数在几何学、物理学、工程学等领域都有重要的应用。常见的三角函数有：\n\n- 正弦函数（sin）：描述直角三角形中一个锐角的对边与斜边的比值。\n- 余弦函数（cos）：描述直角三角形中一个锐角的邻边与斜边的比值。\n- 正切函数（tan）：描述正弦和余弦的比值，也就是对边与邻边的比值。\n- 余切函数（cot）：描述正切的倒数，即邻边与对边的比值。\n- 正割函数（sec）：描述余弦的倒数，即斜边与邻边的比值。\n- 余割函数（csc）：描述正弦的倒数，即斜边与对边的比值。\n\n这些函数在三角学的应用中非常常见，对于解决与角度和三角形相关的问题具有重要作用。'

In [6]:
withMemoryChain.invoke(
  {"input": "为什么叫三角而不是四角?"},
  config={"configurable": {"session_id": "abc123"}},
)

'“三角函数”之所以称为“三角”函数，是因为它们最初是用来描述三角形内角和边长之间的关系的。在欧几里德几何中，三角形是最基本的几何形状之一，由三条边和三个内角组成。\n\n在直角三角形中，我们可以通过三角函数（正弦、余弦、正切等）来描述角度和边长之间的关系。这些函数是基于三角形内角的定义，因此被称为“三角函数”。\n\n虽然“三角函数”这个名称中包含“三角”，但实际上这些函数并不仅限于三角形的应用。它们在数学中的应用非常广泛，可以描述圆的弧度、周期性波动等各种现象，不仅仅局限于三角形。因此，“三角函数”这个名称更多是源自最初应用于三角形的背景，而非仅仅限制于三角形的概念。'

In [7]:
withMemoryChain.invoke(
  {"input": "小学会学到吗?"},
  config={"configurable": {"session_id": "abc123"}},
)

'一般来说，小学阶段的数学课程并不会涉及到三角函数这一较为高级的概念。小学阶段主要侧重于基本的数学概念和技能，如加减乘除、分数、几何图形等。\n\n三角函数通常是在中学阶段的数学课程中开始学习的，通常是在初中或高中的数学课程中出现。学生在学习三角函数之前，通常需要掌握一定的代数知识和几何知识，以便更好地理解和运用三角函数的概念。\n\n因此，三角函数通常不是小学阶段的数学课程内容，而是在更高年级的数学学习中逐渐引入和深入学习的内容。'

In [8]:
memory.shorterm_messages("abc123")

[HumanMessage(content='为什么叫三角而不是四角?'),
 AIMessage(content='“三角函数”之所以称为“三角”函数，是因为它们最初是用来描述三角形内角和边长之间的关系的。在欧几里德几何中，三角形是最基本的几何形状之一，由三条边和三个内角组成。\n\n在直角三角形中，我们可以通过三角函数（正弦、余弦、正切等）来描述角度和边长之间的关系。这些函数是基于三角形内角的定义，因此被称为“三角函数”。\n\n虽然“三角函数”这个名称中包含“三角”，但实际上这些函数并不仅限于三角形的应用。它们在数学中的应用非常广泛，可以描述圆的弧度、周期性波动等各种现象，不仅仅局限于三角形。因此，“三角函数”这个名称更多是源自最初应用于三角形的背景，而非仅仅限制于三角形的概念。'),
 HumanMessage(content='小学会学到吗?'),
 AIMessage(content='一般来说，小学阶段的数学课程并不会涉及到三角函数这一较为高级的概念。小学阶段主要侧重于基本的数学概念和技能，如加减乘除、分数、几何图形等。\n\n三角函数通常是在中学阶段的数学课程中开始学习的，通常是在初中或高中的数学课程中出现。学生在学习三角函数之前，通常需要掌握一定的代数知识和几何知识，以便更好地理解和运用三角函数的概念。\n\n因此，三角函数通常不是小学阶段的数学课程内容，而是在更高年级的数学学习中逐渐引入和深入学习的内容。')]

In [9]:
memory.longterm_messages("abc123")

[HumanMessage(content='三角函数什么意思？?'),
 AIMessage(content='三角函数是指正弦、余弦、正切、余切、正割和余割这六个与直角三角形内角相关的函数。这些函数是数学中重要的概念，用于描述角度和三角形之间的关系。在数学中，三角函数在几何学、物理学、工程学等领域都有重要的应用。常见的三角函数有：\n\n- 正弦函数（sin）：描述直角三角形中一个锐角的对边与斜边的比值。\n- 余弦函数（cos）：描述直角三角形中一个锐角的邻边与斜边的比值。\n- 正切函数（tan）：描述正弦和余弦的比值，也就是对边与邻边的比值。\n- 余切函数（cot）：描述正切的倒数，即邻边与对边的比值。\n- 正割函数（sec）：描述余弦的倒数，即斜边与邻边的比值。\n- 余割函数（csc）：描述正弦的倒数，即斜边与对边的比值。\n\n这些函数在三角学的应用中非常常见，对于解决与角度和三角形相关的问题具有重要作用。'),
 HumanMessage(content='为什么叫三角而不是四角?'),
 AIMessage(content='“三角函数”之所以称为“三角”函数，是因为它们最初是用来描述三角形内角和边长之间的关系的。在欧几里德几何中，三角形是最基本的几何形状之一，由三条边和三个内角组成。\n\n在直角三角形中，我们可以通过三角函数（正弦、余弦、正切等）来描述角度和边长之间的关系。这些函数是基于三角形内角的定义，因此被称为“三角函数”。\n\n虽然“三角函数”这个名称中包含“三角”，但实际上这些函数并不仅限于三角形的应用。它们在数学中的应用非常广泛，可以描述圆的弧度、周期性波动等各种现象，不仅仅局限于三角形。因此，“三角函数”这个名称更多是源自最初应用于三角形的背景，而非仅仅限制于三角形的概念。'),
 HumanMessage(content='小学会学到吗?'),
 AIMessage(content='一般来说，小学阶段的数学课程并不会涉及到三角函数这一较为高级的概念。小学阶段主要侧重于基本的数学概念和技能，如加减乘除、分数、几何图形等。\n\n三角函数通常是在中学阶段的数学课程中开始学习的，通常是在初中或高中的数学课程中出现。学生在学习三角函数之前，通常需要掌握一定的代数知识和几何知识，以便更好地理解和运用三角函数的概念。\n\n因此，三

In [10]:
for chunk in withMemoryChain.stream(
  {"input": "我最初的问题是什么来着?"},
  config={"configurable": {"session_id": "abc123"}},
):
  print(chunk, end="|", flush=True)

|你|最|初|的|问题|是|关|于|为|什|么|称|为|“|三|角|”|函数|而|不|是|“|四|角|”|函数|的|。||

# redis持久化

```shell
pip install -U langchain-community redis
```

创建窗口管理器时要指定redis的工厂函数，其他与上面的例子相同。

In [ ]:
from langchain_chinese import MemoryManager
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory
from langchain_community.chat_message_histories import RedisChatMessageHistory

window = ConversationBufferWindowMemory(
    return_messages=True, k=2, chat_memory = ChatMessageHistory()
)

memory = MemoryManager(
    lambda session_id: RedisChatMessageHistory(
        session_id, url="redis://localhost:6379"
    ),
    shorterm_memory = window
)